In [ ]:
import pyvista as pv

from kale import Engine, time_controls
from kale.algorithms import contour_banded

In [ ]:
# If you can check out https://github.com/pyvista/pyvista/pull/3385, use:
await pv.set_jupyter_backend("server")

## Load Data

Set up the driver of the data IO and time step management with the `kale.Engine` class

In [ ]:
engine = Engine("2022_11_30_11_52_49_mesh_geometry.vtk", "2022_11_30_11_52_49.hdf")
# View available data arrays
engine.keys

In [ ]:
# Set which array to be active
engine.mesh.active_scalars_name = "cumulative_slip"
# Initialize time where data look interesting
engine.time_step = 55849

## Plot with time controls

In [ ]:
pl = pv.Plotter()
pl.add_mesh(
    engine.algorithm,
    cmap="CET_C3",
    clim=engine.clim(),
)
pl.add_mesh(engine.boundary, color="k", line_width=3)
pl.show()

In [ ]:
controls = time_controls(engine, pl)
controls

## Banded Contouring

In [ ]:
n_contours = 20

In [ ]:
contour, edges = contour_banded(
    engine.algorithm,
    n_contours,
    rng=engine.clim(),
    scalars=engine.mesh.active_scalars_name,
)

In [ ]:
pl = pv.Plotter()
pl.add_mesh(engine.boundary, color="k", line_width=3)
pl.add_mesh(
    contour,
    cmap="CET_C3",
    clim=engine.clim(),
    scalars="cumulative_slip",
    n_colors=n_contours - 1,
)
pl.add_mesh(edges.GetOutputPort(1), color="k", line_width=3)
pl.show()

In [ ]:
controls = time_controls(engine, pl)
controls

### 4-panel view (all variables)

This still needs some optimization to make look really nice but here is a start

In [ ]:
engine.keys

In [ ]:
pl = pv.Plotter(shape=(1, 4), window_size=(1024 * 2, 768))

for i, var in enumerate(engine.keys):
    pl.subplot(0, i)
    contour, edges = contour_banded(
        engine.algorithm, n_contours, rng=engine.clim(), scalars=var
    )
    pl.add_mesh(contour, cmap="CET_C3", clim=engine.clim(var), n_colors=n_contours - 1)
    pl.add_mesh(edges.GetOutputPort(1), color="k", line_width=3)
    pl.add_mesh(engine.boundary, color="k", line_width=3)

pl.link_views()
pl.show()

In [ ]:
time_controls(engine, pl)